# Collect teams' ranking and matches history data

## Teams' ranking points

In [1]:
from requests import get
from bs4 import BeautifulSoup
from pandas import DataFrame

names = []
pts = []

for page in range(1, 7):
    soup = BeautifulSoup(get('https://www.gosugamers.net/dota2/rankings/list?page=' + str(page)).content, 'html.parser')
    names = names + list(filter(None, [flag.next_sibling[30:-29] for flag in soup.find_all('span', class_='flag')]))
    pts = pts + [int(pt.get_text()[33:-29]) for pt in soup.find_all('span', class_='elo')]

frame = DataFrame()
frame['name'] = names
frame['pt'] = pts

In [3]:
# from selenium import webdriver

# options = webdriver.ChromeOptions()
# options.add_argument('--ignore-certificate-errors')
# options.add_argument('--incognito')
# options.add_argument('--headless')
# driver = webdriver.Chrome(chrome_options=options)
# driver.get("https://www.gosugamers.net/dota2/rankings")
# page_source = driver.page_source

In [4]:
# from bs4 import BeautifulSoup

# soup = BeautifulSoup(page_source, 'lxml')